In [1]:
import os 

In [2]:
%pwd

'/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision'

In [8]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class PreaperBaseModelConfig:
    root_dir : Path 
    base_model_path: Path
    update_base_model_path: Path
    params_model_name : str 
    params_augmentation : bool 
    params_image_size : list
    params_batch_size : int
    params_include_top : bool
    params_epochs : int
    params_num_classes : int
    params_weights : str 
    params_learning_rate : float



In [6]:
from srcTrackVision.constants import *
from srcTrackVision.utils.common import read_yaml , create_directories

In [50]:
class ConfigurationManager:
    def __init__(
            self , 
            config_filepath = '/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/config/config.yaml',
            params_filepath = '/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/params.yaml' ):
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # Create the root dir 
        create_directories([self.config.artifacts_root])


    def get_preaper_base_mode_config(self) -> PreaperBaseModelConfig:
        config = self.config.preaper_base_model
        create_directories([config.root_dir])
        preaper_base_model_config = PreaperBaseModelConfig(
            root_dir= Path(config.root_dir),
            base_model_path= Path(config.base_model_path),
            update_base_model_path = Path(config.update_base_model_path),
            params_model_name= self.params.MODEL_NAME,
            params_augmentation= self.params.AUGMENTATION,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE,
            params_include_top= self.params.INCLUDE_TOP,
            params_epochs= self.params.EPOCHS,
            params_num_classes= self.params.CLASSES,
            params_weights= self.params.WEIGHTS,
            params_learning_rate= self.params.LEARNING_RATE,
             

        )

    
        return preaper_base_model_config

In [111]:
from keras_vggface.vggface import  VGGFace
import tensorflow as tf

class PreaperBaseModel():
    def __init__ (self  , config = PreaperBaseModelConfig):
        self.config = config


    def get_base_model (self):
        self.model = VGGFace(model =  self.config.params_model_name 
                             , include_top= self.config.params_include_top
                             , weights= self.config.params_weights
                             , classes= self.config.params_num_classes
                              ,input_shape= self.config.params_image_size
                              
                              
        )
        self.save_model(path = self.config.base_model_path , model = self.model)
    @staticmethod
    def _prepare_base_model (model , classes , freez_all  , freez_taill  , unfreez_tail , learning_rate):
        if freez_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freez_taill is not None) and (freez_taill > 0):
            for layer in model.layers[:-unfreez_tail]:
                            layer.trainable = False
                            
                             
        elif (unfreez_tail is not None) and (unfreez_tail > 0):
            for layer in model.layers[-unfreez_tail:]:
                 layer.trainable = True
                 

        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        dense_in = tf.keras.layers.Dense(units = 128, activation= 'relu')(flatten_in)
        dropout_in = tf.keras.layers.Dropout(rate = 0.5)(dense_in)
        prediction_out = tf.keras.layers.Dense(units = classes
                                                , activation ='softmax')(dropout_in)

        full_model = tf.keras.models.Model(inputs = model.input, outputs = prediction_out)

        full_model.compile(
              optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
              , loss = tf.keras.losses.CategoricalCrossentropy()
              , metrics = ['accuracy']
        )
        full_model.summary()
         

        return full_model

    def update_base_model (self):
          self.full_model = self._prepare_base_model(
                model = self.model
                , classes = self.config.params_num_classes
                , freez_all = True 
                , freez_taill = None
                , unfreez_tail = None
                , learning_rate = self.config.params_learning_rate
          )
          self.save_model(path = self.config.update_base_model_path , model = self.full_model)
    @staticmethod
    def save_model (path : Path , model : tf.keras.models.Model):
          model.save(path)
          

In [112]:
from srcTrackVision import logger
try:
    config = ConfigurationManager()
    preaper_base_model_config = config.get_preaper_base_mode_config()
    prepaper_base_model = PreaperBaseModel(config= preaper_base_model_config)
    prepaper_base_model.get_base_model()
    prepaper_base_model.update_base_model()


except Exception as e:
    logger.exception(e)
    raise e

2023-06-17 20:21:52,182 - trackvisionlogger - INFO - yaml file /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/config/config.yaml loaded successfully
2023-06-17 20:21:52,187 - trackvisionlogger - INFO - yaml file /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/params.yaml loaded successfully
2023-06-17 20:21:52,189 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts
2023-06-17 20:21:52,190 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts/preaper_base_model
2023-06-17 20:21:53,799 - tensorflow - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2023-06-17 20:21:54,081 - absl - WARNING - At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer inste